In [5]:
import timeit
from collections import deque
from array import array
import numpy as np

# Benchmark setup
def benchmark_list_operations(n):
    lst = []
    for i in range(n):
        lst.insert(0, i)  # insert at the beginning
    for i in range(n):
        lst.pop(0)  # pop from the beginning

def benchmark_deque_operations(n):
    dq = deque()
    for i in range(n):
        dq.appendleft(i)  # insert at the beginning
    for i in range(n):
        dq.popleft()  # pop from the beginning

def benchmark_array_operations(n):
    arr = array('i', [])
    for i in range(n):
        arr.insert(0, i)  # insert at the beginning
    for i in range(n):
        arr.pop(0)  # pop from the beginning

# Simulate "insertion at the beginning" with numpy using shifting
def benchmark_numpy_operations_shift(n):
    arr = np.zeros(n, dtype=np.int32)
    for i in range(n):
        arr = np.roll(arr, 1)  # Shift elements to the right
        arr[0] = i  # Insert at the beginning
    for i in range(n):
        arr = np.roll(arr, -1)  # Shift elements to the left

# Benchmarking
n = 10000
time_list = timeit.timeit(lambda: benchmark_list_operations(n), number=1)
time_deque = timeit.timeit(lambda: benchmark_deque_operations(n), number=1)
time_array = timeit.timeit(lambda: benchmark_array_operations(n), number=1)
time_numpy_shift = timeit.timeit(lambda: benchmark_numpy_operations_shift(n), number=1)

time_list, time_deque, time_array, time_numpy_shift


(0.008278004010207951,
 0.00026246701600030065,
 0.0026771900011226535,
 0.07825820901780389)

In [6]:
import timeit

# Benchmark functions
def benchmark_list_read(n):
    lst = [i for i in range(n)]
    for i in range(n):
        x = lst[i]

def benchmark_list_insert(n):
    lst = []
    for i in range(n):
        lst.insert(0, i)

def benchmark_list_append(n):
    lst = []
    for i in range(n):
        lst.append(i)

def benchmark_list_pop(n):
    lst = [i for i in range(n)]
    for i in range(n):
        x = lst.pop()

def benchmark_list_remove(n):
    lst = [i for i in range(n)]
    for i in range(n):
        lst.remove(i)

def benchmark_list_index(n):
    lst = [i for i in range(n)]
    for i in range(n):
        x = lst.index(i)

# Benchmarking
n = 10000
time_list_read = timeit.timeit(lambda: benchmark_list_read(n), number=5)
time_list_insert = timeit.timeit(lambda: benchmark_list_insert(n), number=5)
time_list_append = timeit.timeit(lambda: benchmark_list_append(n), number=5)
time_list_pop = timeit.timeit(lambda: benchmark_list_pop(n), number=5)
time_list_remove = timeit.timeit(lambda: benchmark_list_remove(n), number=5)
time_list_index = timeit.timeit(lambda: benchmark_list_index(n), number=5)

print('Read:', time_list_read)
print('Insert:', time_list_insert)
print('Append:', time_list_append)
print('Pop:', time_list_pop)
print('Remove:', time_list_remove)
print('Index:', time_list_index)


Read: 0.0009538560116197914
Insert: 0.021579102001851425
Append: 0.0005801899824291468
Pop: 0.0010511180153116584
Remove: 0.018988015013746917
Index: 0.9868873929954134


In [7]:
# Benchmark functions
def benchmark_set_add(n):
    s = set()
    for i in range(n):
        s.add(i)

def benchmark_set_remove(n):
    s = {i for i in range(n)}
    for i in range(n):
        s.remove(i)

def benchmark_set_membership(n):
    s = {i for i in range(n)}
    for i in range(n):
        x = i in s  # Membership test

def benchmark_set_discard(n):
    s = {i for i in range(n)}
    for i in range(n):
        s.discard(i)

def benchmark_set_clear(n):
    s = {i for i in range(n)}
    s.clear()

def benchmark_set_union(n):
    s1 = {i for i in range(n)}
    s2 = {i for i in range(n, 2*n)}
    s = s1.union(s2)

def benchmark_set_intersection(n):
    s1 = {i for i in range(n)}
    s2 = {i for i in range(int(n/2), n + int(n/2))}
    s = s1.intersection(s2)

def benchmark_set_difference(n):
    s1 = {i for i in range(n)}
    s2 = {i for i in range(int(n/2), n + int(n/2))}
    s = s1.difference(s2)

# Benchmarking
n = 10000
time_set_add = timeit.timeit(lambda: benchmark_set_add(n), number=100)
time_set_remove = timeit.timeit(lambda: benchmark_set_remove(n), number=100)
time_set_membership = timeit.timeit(lambda: benchmark_set_membership(n), number=100)
time_set_discard = timeit.timeit(lambda: benchmark_set_discard(n), number=100)
time_set_clear = timeit.timeit(lambda: benchmark_set_clear(n), number=100)
time_set_union = timeit.timeit(lambda: benchmark_set_union(n), number=100)
time_set_intersection = timeit.timeit(lambda: benchmark_set_intersection(n), number=100)
time_set_difference = timeit.timeit(lambda: benchmark_set_difference(n), number=100)

# Results
print('Add:', time_set_add)
print('Remove:', time_set_remove)
print('Membership:', time_set_membership)
print('Discard:', time_set_discard)
print('Clear:', time_set_clear)
print('Union:', time_set_union)
print('Intersection:', time_set_intersection)
print('Difference:', time_set_difference)

Add: 0.016842254990478978
Remove: 0.03229504299815744
Membership: 0.03208695098874159
Discard: 0.031928241020068526
Clear: 0.012840510986279696
Union: 0.07677349197911099
Intersection: 0.03397759099607356
Difference: 0.03358016998390667
